In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv("titanic.csv")

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df.drop(columns = ["PassengerId","Name","Ticket","Cabin"], inplace = True)

In [7]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


STEP 1 - TRAIN_TEST_SPLIT

In [9]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=["Survived"]),df["Survived"],test_size = 0.2, random_state=42)

In [10]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [11]:
y_train.head()

331    0
733    0
382    0
704    0
813    0
Name: Survived, dtype: int64

### Applying imputation

to be continued 

### sklearn using pipeline 
1- Impute missing values 
2- OHE
3- Scaling 
4- Feature selection 
5- Model train 

In [31]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest ,chi2
from sklearn.tree import DecisionTreeClassifier

### LETS START

In [13]:
df = pd.read_csv("titanic.csv")

In [14]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
df.drop(["PassengerId","Name","Ticket","Cabin"],axis = 1 , inplace = True)

In [17]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


### STEP 2
Train test split

In [23]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(["Survived"], axis = 1),
                                                 df["Survived"],
                                                 test_size = 0.2,
                                                 random_state = 42)

In [25]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [27]:
y_train.shape

(712,)

lets do the transformer

### IMPUTER TRANFORMER 

In [80]:
trf1 = ColumnTransformer([
    ("impute_age", SimpleImputer(),[2]),
    ("impute_embark", SimpleImputer(strategy = "most_frequent"),[6])
],remainder = 'passthrough')

## ONE HOT ENCODER 

In [79]:
trf2 = ColumnTransformer([
    ("ohe_sex_embarked", OneHotEncoder(sparse_output = False, handle_unknown="ignore"),[1,6])
    
],remainder = 'passthrough')

### Scaling

In [77]:
trf3 = ColumnTransformer([
    ("scale", MinMaxScaler(),slice (0,10)) 
])

### FEATURE SELECTION

In [70]:
trf4 = SelectKBest(score_func = chi2,k=8)

## Train the model

In [71]:
trf5 = DecisionTreeClassifier()


### CREATE PIPELINE

In [81]:
pipe = Pipeline([
    ("trf1", trf1),
    ("trf2", trf2),
    ("trf3", trf3),
    ("trf4", trf4),
    ("trf5", trf5)
    
])

## PipeLine VS Make_pipeline

Pipeline requires naming of steps, make_pipeline does not 

(same applies to Columntranformer vs make_column_tranformer)

In [82]:
#ALTERNATIVE SYNTAX
pipe2 = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [83]:
#TRAIN

In [84]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embark',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x00000262D24C9080>)),
                ('trf5', DecisionTreeClassifier())])